In [ ]:
# https://colab.research.google.com/drive/1uSlWtJdZmLrI3FCNIlUHFxwAJiSu2J0-#scrollTo=-ONLrgJK99TQ
!pip install transformers
!pip install statsmodels

### Models to run

bert-large-cased-whole-word-masking-finetuned-squad  
bert-large-uncased-whole-word-masking-finetuned-squad  
distilbert-base-cased-distilled-squad  
distilbert-base-uncased-distilled-squad  


In [68]:
from transformers import BertForQuestionAnswering, AutoModelForQuestionAnswering, AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from transformers import BertTokenizer
import torch
import torch.nn as nn
import json

model_type = 'distilbert_uncased'

if model_type == 'bert_cased':
    huggingface_model_name = 'bert-large-cased-whole-word-masking-finetuned-squad'
    model = BertForQuestionAnswering.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad',
                                                    cache_dir="/data/models_cache")
    tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad',
                                             cache_dir="/data/models_cache")

elif model_type == 'bert_uncased':
    huggingface_model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad',
                                                    cache_dir="/data/models_cache")
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad',
                                             cache_dir="/data/models_cache")
    
elif model_type == 'distilbert_cased':
    huggingface_model_name = 'distilbert-base-cased-distilled-squad'
    model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad",
                                                    cache_dir="/data/models_cache")
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad",
                                             cache_dir="/data/models_cache")
    
elif model_type == 'distilbert_uncased':
    huggingface_model_name = 'distilbert-base-uncased-distilled-squad'
    model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad",
                                                    cache_dir="/data/models_cache")
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad",
                                             cache_dir="/data/models_cache")


In [69]:
# Grab 2 GPU's if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
  model = nn.DataParallel(model, device_ids=[0,1])

model = model.to(device)

In [70]:


amazon_qa_json = '/data/distribution_shift/new_qa/amazon_reviews_v1.0.json'

def check_answer(answer, ground_truths):
    for ground_truth in ground_truths:
        if answer.lower() == ground_truth['text'].lower():
            return True
    return False

def predict_answer(question, context):
    input_ids = tokenizer.encode(question, context, max_length=512)

    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

#    start_scores, end_scores = model(torch.tensor([input_ids]).to(device), # The tokens representing our input text.
#                      token_type_ids=torch.tensor([segment_ids]).to(device)) # The segment IDs to differentiate question from answer_text

    start_scores, end_scores = model(torch.tensor([input_ids]).to(device))
    
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Combine the tokens in the answer and print it out.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    #answer = ' '.join(tokens[answer_start:answer_end+1])
    
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    
    return answer, answer_start

## JSON Format
# -data
#   -paragraphs
#     -context
#     -qas
#       -question
#       -id
#       -answers
#          -text
#          -answer_start
#   -title
#   -split

results = {}
count = 0
with open(amazon_qa_json) as json_file:
    amazon_qa = json.load(json_file)['data']
    for splits in amazon_qa:
        print(splits['title'])
        print(splits['split'])
        
        paragraphs = splits['paragraphs']
        for paragraph in paragraphs:
            count += 1
            #print(paragraph['context'])
            
            for qa in paragraph['qas']:
                print(qa['question'])
                
                # Predict the anser, and then validate against ground truth. 
                answer, answer_start = predict_answer(qa['question'], paragraph['context'])
                result = check_answer(answer, qa['answers'])
                
                #print("Acceptable answers: {}".format(qa['answers']))
                print('Answer: "{}": Answer_Start:{}  Result:{}'.format(answer, answer_start, result))
                results.update([(qa['id'], answer)])
            print("\n\n\n")

    

Amazon_Reviews_2030
amazon_reviews
What size is the holder?
Answer: "not too big": Answer_Start:16  Result:False
What does it fit nicely?
Answer: "lipstick": Answer_Start:27  Result:False
Does the writer recommend the holder?
Answer: "i would highly recommend": Answer_Start:63  Result:False
Do you have to dig through a makeup bag?
Answer: "not have to dig through a makeup bag anymore": Answer_Start:56  Result:False
What is the product?
Answer: "lipstick , lip gloss": Answer_Start:26  Result:False




Who assembled the glass shelves?
Answer: "my wife": Answer_Start:20  Result:True
What's on the second shelf?
Answer: "my dish network 722": Answer_Start:92  Result:False
What's on the bottom shelf?
Answer: "an old ken wood stereo receive": Answer_Start:117  Result:False
What are the dimensions of the receiver?
Answer: "17 " w x 14 " d x 5 " h": Answer_Start:146  Result:False




Does the writer recommend the pod?
Answer: "don": Answer_Start:93  Result:False
Did several pods make a differen

Answer: "a mesh see - through material": Answer_Start:29  Result:False
What material is the storage bag made from?
Answer: "mesh see - through": Answer_Start:26  Result:False
What sort of loop is at the top?
Answer: "velcro": Answer_Start:66  Result:True
Does the bag function as required?
Answer: "it still does what it ' s supposed to , so no complaints": Answer_Start:47  Result:False
Where might one hang the bag?
Answer: "the bar under the shelf in my closet": Answer_Start:79  Result:False




Does the cooker poach eggs evenly?
Answer: "this cooker cannot do that": Answer_Start:68  Result:False
Does the cooker work well for boiling eggs?
Answer: "this cooker cannot do that": Answer_Start:69  Result:False
What is the main complaint about the cooker's ability to poach eggs?
Answer: "cooker cannot do that . you put the eggs in the poaching tray , and the top part of the egg ( that is exposed to steam": Answer_Start:77  Result:False
Where do you put the eggs to poach them?
Answer: "the po

Answer: "the resistance": Answer_Start:84  Result:False
What don't you have to do to the handles?
Answer: "turn the handle": Answer_Start:71  Result:False
Is it easy or difficult to turn the handle?
Answer: "easy": Answer_Start:120  Result:True




What sort of grip do these hangers use to hold onto a pair of slacks?
Answer: "vice - like": Answer_Start:45  Result:False
What has a rubbery padding at the tips?
Answer: "pincer - like holders": Answer_Start:61  Result:False
How many pairs on a hanger cause the grips to loosen on the hanging mechanism over time?
Answer: "two": Answer_Start:146  Result:True
Where are the hangers found?
Answer: "amazon": Answer_Start:178  Result:True
Where would it be a hassle t find these hangers?
Answer: "amazon": Answer_Start:184  Result:False




Are they more likely freezing or canning the contents of the ball jars?
Answer: "freezing the contents than canning": Answer_Start:69  Result:False
What doesn't the food processor have that makes it difficult tra

Answer: "queen size platform bed": Answer_Start:11  Result:True
How well has the bed held up?
Answer: "fairly well": Answer_Start:57  Result:True
How many times did he have to take it apart?
Answer: "3": Answer_Start:29  Result:True
What does the author use the bottom drawers for?
Answer: "personal items": Answer_Start:116  Result:True
Why would the author recommend it to other people?
Answer: "price and durability": Answer_Start:132  Result:True




What did this one replace?
Answer: "a much cheaper": Answer_Start:10  Result:False
Where did the author decided to put the laundry basket?
Answer: "under the blazers": Answer_Start:139  Result:True
How long the old one last him?
Answer: "five years": Answer_Start:20  Result:True
What part is the other still waiting for?
Answer: "cover": Answer_Start:50  Result:True
What is the bottom shelf made out of?
Answer: "cardboard": Answer_Start:119  Result:True




What was wrong with the pan when he first got it?
Answer: "rust": Answer_Start:43  R

Answer: "more expensive": Answer_Start:45  Result:False
Where do i put the Thermoworks?
Answer: "fast , foldable thermoworks and the fast , non - foldable": Answer_Start:17  Result:False




What is the price of the repair to the mixer?
Answer: "$ 129": Answer_Start:17  Result:False
What was the mixer used for?
Answer: "to make 4 loaves of bread / week": Answer_Start:75  Result:False
What happened to the first mixer?
Answer: "it died while making cookie dough . this is my second 600 - series mixer ; kitchenaid ( to their credit ) swapped out the first under warranty after it started behaving erratically": Answer_Start:29  Result:False
How much was the mixer in use for?
Answer: "to make 4 loaves of bread / week": Answer_Start:77  Result:False
What happened to the second mixer?
Answer: "it died while making cookie dough": Answer_Start:29  Result:False
What is the price of the repair to the mixer?
Answer: "$ 129": Answer_Start:17  Result:False
What happened to the first mixer?
Answer: "it

...

Answer: "tuna": Answer_Start:136  Result:True




What is the capacity of the bottom space?
Answer: "10 > 16oz": Answer_Start:67  Result:False
Is the image accurate for space?
Answer: "it holds a lot more than th picture implies": Answer_Start:113  Result:True
Does it have quality construction?
Answer: "it": Answer_Start:130  Result:False
Is it easy to damage?
Answer: "it": Answer_Start:130  Result:False
How much can the top section contain?
Answer: "10 > 16oz": Answer_Start:66  Result:False
What have they struggled with for years?
Answer: "space": Answer_Start:14  Result:False
What do they use it to hold?
Answer: "cans of tuna , small jars of preserves , toothpicks and etc": Answer_Start:99  Result:False
What does it do more than the picture implies?
Answer: "it holds a lot more than th picture implies , which is a good thing my book": Answer_Start:116  Result:False
How is the plastic rated?
Answer: "10 > 16oz": Answer_Start:64  Result:False




What is considered to be problematic?
A

Answer: "amazon": Answer_Start:112  Result:False
Where did the reviewer take a vacation?
Answer: "alaskan cruise": Answer_Start:32  Result:True
What kind of kitchen utensil is being reviewed?
Answer: "knife": Answer_Start:54  Result:True
Which relative asked the reviewer to buy a gift?
Answer: "my mom": Answer_Start:93  Result:False
Where did the reviewer purchase this item?
Answer: "amazon": Answer_Start:110  Result:True




Does it have a switch?
Answer: "it does not have an on / off switch": Answer_Start:14  Result:False
What star rating does the reviewer give the product?
Answer: "four": Answer_Start:40  Result:True
What kind of appliance is being reviewed?
Answer: "popcorn popper": Answer_Start:63  Result:True
Where did the person search for products to buy?
Answer: "amazon": Answer_Start:79  Result:True
How long does the appliance need to heat up before using?
Answer: "a few minutes": Answer_Start:109  Result:True




What kind of mats are these?
Answer: "chopping mats": Answer_S

Answer: "20 mins": Answer_Start:39  Result:True
What can cause it to feel like it will tip?
Answer: "so light , when you hang heavy towels": Answer_Start:81  Result:False
What are a great idea and made of plastic?
Answer: "sock clips": Answer_Start:116  Result:True
What is a bit small on the holder?
Answer: "space between each rail": Answer_Start:138  Result:True




Amazon_Reviews_1165
amazon_reviews
What product was being reviewed?
Answer: "[CLS]": Answer_Start:0  Result:False
The casing is made of what type of material?
Answer: "rubber / plastic": Answer_Start:70  Result:False
Why did the reviewer take off on star from the rating?
Answer: "because of one design or manufacturing issue": Answer_Start:183  Result:True
Did the French Press meet all the reviewers expectations?
Answer: "this item met all our expectations": Answer_Start:249  Result:True
Is disassembly and cleaning easy?
Answer: "it is quite easy to disassemble and clean": Answer_Start:138  Result:False




What product is 

Answer: "danielle": Answer_Start:20  Result:True
The name of the cookbook mentioned is?
Answer: "against all grain cookbook": Answer_Start:13  Result:False
The product bought was used to?
Answer: "spaghetti noodles": Answer_Start:30  Result:False
The item is the favored item to use where?
Answer: "the kitchen": Answer_Start:42  Result:False




Amazon_Reviews_1450
amazon_reviews
From what online ecommerce service were the filters purchased from?
Answer: "amazon": Answer_Start:307  Result:True
What is the brand name of the tester to check water quality?
Answer: "intellitec": Answer_Start:34  Result:True
What is the max rating from the EPA for bottled water?
Answer: "500": Answer_Start:165  Result:True
What is the brand name of the reverse osmosis filtration system purchased?
Answer: "ro": Answer_Start:28  Result:False
Who was the Amazon seller the filters were purchased from?
Answer: "ispring": Answer_Start:311  Result:True
From what online ecommerce service were the filters purchased f

Answer: "christmas": Answer_Start:23  Result:True
what actor is the writer a fan of?
Answer: "john wayne": Answer_Start:33  Result:True
what calendar does the writer look at for motivation?
Answer: "john wayne": Answer_Start:45  Result:False
what product is the writer reviewing?
Answer: "great coffe mug": Answer_Start:186  Result:False




what product is the writer reviewing?
Answer: "pressure cooking": Answer_Start:40  Result:False
what brand of product is the writer reviewing?
Answer: "pressure cooking": Answer_Start:42  Result:False
what kind of diet is the writer on?
Answer: "plant - based": Answer_Start:83  Result:False
how long does it take to cook pinto beans in this fryer?
Answer: "three easy minutes": Answer_Start:111  Result:True
what material is this product made from?
Answer: "stainless": Answer_Start:137  Result:True




Amazon_Reviews_540
amazon_reviews
what did they want to run for 2 hours?
Answer: "fountain": Answer_Start:98  Result:False




Why was the product first 

In [71]:
predictions_filename = './output/'+huggingface_model_name+'-predictions.json'
with open(predictions_filename, 'w') as fp:
    json.dump(results, fp)

In [72]:
from eval_v1_1 import evaluate

with open(amazon_qa_json) as json_file:
    amazon_qa = json.load(json_file)['data']
    
evaluate(amazon_qa, results)

{'exact_match': 51.78553363682347,
 'exact_match_ci': (50.79503391873498, 52.77498080096132),
 'f1': 67.1402286661281,
 'f1_ci': (66.33834793758437, 67.94210939467114)}